# The maximum entropy principle meets optimality theory in phonology

In this homework we will replicate Goldwater and Johnson's 2003 maxent OT analysis of Wolof.  The Wolof data capture a categorical rather than gradient pattern, and so could also be analyzed using standard (number-less) optimality theory, i.e. in terms of strictly ranked constraints.  The advantages of instead analyzing the data using maxent OT are: (1) that the same method can be applied to non-categorical, gradient patterns, and (2) that  constraint importance is learned automatically and reliably, including in the case of noisy data.

These two advantages flow naturallly from the adoption of a probabilistic formulation of the problem.

The Wolof data we will work with are from Boersma (1999), and are bundled with the [Praat](https://www.fon.hum.uva.nl/praat/) software package.  The data have been extracted from Praat for you, and annotated, and are supplied with this homework.  There is also an associated PDF file from Praat which shows the tableaux in the usual OT format.

What remains is to:

- Read in the data, and manipulate it into the desired dataframe format.
- Spot-check the formatted data against the PDF of tableaux that it is supposed to correspond to.
- Construct a maxent classifier in line with G&J's description.
- Train the classifier on the data.
- Examine the results to see whether they align with G&J's reported results.



In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

### Read in data

The data contain OT tableaux for the Wolof tongue-root grammar described in Boersma (1999), which includes five constraints:

- <span style="font-variant:small-caps;">*RtrHi</span> : High vowels must not have a retracted tongue root (rtr).
- <span style="font-variant:small-caps;">*AtrLo</span> : Low vowels must not have an advanced tongue root (atr).
- <span style="font-variant:small-caps;">Parse[Rtr]</span> : If an input segment is [rtr], it must be realized as [rtr] in the output.
- <span style="font-variant:small-caps;">Parse[Atr]</span> : If an input segment is [atr], it must be realized as [atr] in the output.
- <span style="font-variant:small-caps;">*Gesture[Contour]</span> : Do not change from [rtr] to [atr], or vice versa, within a word.

We first read in the data, and convert it to a list of dataframes, one per tableau.

We then reformat those tableau dataframes to match the format in the associated PDF file, wolof-tableaux-aug-15.pdf, and compare the read-in formatted data to the data in the PDF, to make sure we have it correctly.

In [2]:
# start with an empty list.
# we're calling this a corpus, even though more accurately it is a list of tableaux.
corpus = []

# read in corpus from file.  
# corpus is a list of tableaux,
# where each tableau is a dict with keys:
#   input: input form
#   outputs: list of output forms
#   CVC: list of lists of CVCs = constraint violation counts.  
#       outer list is 1 item per output, inner list holds CVCs for each constraint, for this input/output pair
#   winner: index of winning output form
with open("wolof-clean-v3.txt") as f:
    for line in f:
        l = line.strip(" \n")  # nix leading/trailing spaces, newlines.
        if l.startswith("input"):
            d = dict()  # create new tableau entry
            d['input'] = l.split()[2].strip("\"")  # get input form
            d['outputs'] = []
            d['CVC'] = []
        elif l.startswith("winner"):
            d['winner'] = int(l.split()[1])  # get index of winning outputu   
        elif l.startswith("candidate"):
            elts = l.split()
            d['outputs'].append(elts[2].strip("\""))  # get output form, store
            cvc = elts[3:]
            cvc = [int(i) for i in cvc]
            d['CVC'].append(cvc)
        elif l.startswith("end"):
            corpus.append(d)
        else:
            pass  # ignore everything else, including comments

# take a look at the first tableau in dict form.
print(corpus[0])

# tdf = tableaux in df form.  this will be a list of dfs, one per tableau.
tdf = [pd.DataFrame(elt) for elt in corpus]
# now re-examine the first tableau, now in df form.
display(tdf[0])

{'input': 'iti', 'outputs': ['iti', 'ɪti', 'itɪ', 'ɪtɪ'], 'CVC': [[0, 0, 0, 0, 0], [1, 0, 0, 1, 1], [1, 0, 0, 1, 1], [2, 0, 0, 2, 0]], 'winner': 0}


,input,outputs,CVC,winner
0,iti,iti,"[0, 0, 0, 0, 0]",0
1,iti,ɪti,"[1, 0, 0, 1, 1]",0
2,iti,itɪ,"[1, 0, 0, 1, 1]",0
3,iti,ɪtɪ,"[2, 0, 0, 2, 0]",0


In [3]:
# now explode out the CVC column into separate columns, one per constraint.
# rearrange the constraints so they are in the order shown in the PDF - which, confusingly,
# is not the same order as in the input file.  sorry, not my idea.
# then compare your formatted data to that in the PDF.

# constraints, in order of appearance in input file, are: 
in_constraints = ['*RtrHi', '*AtrLo', 'Parse[Rtr]', 'Parse[Atr]', '*Gesture[Contour]']

# constraints, in order of appearance in accompanying PDF file, are:
pdf_constraints = ['*RtrHi', 'Parse[Rtr]', '*Gesture[Contour]', 'Parse[Atr]', '*AtrLo']

# save tdf as original_tdf, and prepare a new tdf variable to hold the expanded tableau dfs.
original_tdf = tdf
tdf = []

for df in original_tdf:
    # create df just for constraints in this tableau, explode them out into separate columns
    cdf = pd.DataFrame(df["CVC"].to_list(), columns=in_constraints)
    # now reorder those constraint columns to be in the order shown in the accompanying pdf
    cdf = cdf[pdf_constraints]
    df = pd.merge(df[['input','winner','outputs']],cdf, left_index=True, right_index=True)
    tdf.append(df)

In [4]:
# print out each tableau in tdf, to spot-check against PDF.
for t in tdf:
    display(t)

,input,winner,outputs,*RtrHi,Parse[Rtr],*Gesture[Contour],Parse[Atr],*AtrLo
0,iti,0,iti,0,0,0,0,0
1,iti,0,ɪti,1,0,1,1,0
2,iti,0,itɪ,1,0,1,1,0
3,iti,0,ɪtɪ,2,0,0,2,0


,input,winner,outputs,*RtrHi,Parse[Rtr],*Gesture[Contour],Parse[Atr],*AtrLo
0,ite,0,ite,0,0,0,0,0
1,ite,0,ɪte,1,0,1,1,0
2,ite,0,itɛ,0,0,1,1,0
3,ite,0,ɪtɛ,1,0,0,2,0


,input,winner,outputs,*RtrHi,Parse[Rtr],*Gesture[Contour],Parse[Atr],*AtrLo
0,itə,0,itə,0,0,0,0,1
1,itə,0,ɪtə,1,0,1,1,1
2,itə,0,ita,0,0,1,1,0
3,itə,0,ɪta,1,0,0,2,0


,input,winner,outputs,*RtrHi,Parse[Rtr],*Gesture[Contour],Parse[Atr],*AtrLo
0,itɪ,2,itɪ,1,0,1,0,0
1,itɪ,2,ɪtɪ,2,0,0,1,0
2,itɪ,2,iti,0,1,0,0,0
3,itɪ,2,ɪti,1,1,1,1,0


,input,winner,outputs,*RtrHi,Parse[Rtr],*Gesture[Contour],Parse[Atr],*AtrLo
0,itɛ,0,itɛ,0,0,1,0,0
1,itɛ,0,ɪtɛ,1,0,0,1,0
2,itɛ,0,ite,0,1,0,0,0
3,itɛ,0,ɪte,1,1,1,1,0


...

,input,winner,outputs,*RtrHi,Parse[Rtr],*Gesture[Contour],Parse[Atr],*AtrLo
0,ate,2,ate,0,0,1,0,0
1,ate,2,əte,0,1,0,0,1
2,ate,2,atɛ,0,0,0,1,0
3,ate,2,ətɛ,0,1,1,1,1


,input,winner,outputs,*RtrHi,Parse[Rtr],*Gesture[Contour],Parse[Atr],*AtrLo
0,atə,2,atə,0,0,1,0,1
1,atə,2,ətə,0,1,0,0,2
2,atə,2,ata,0,0,0,1,0
3,atə,2,əta,0,1,1,1,1


,input,winner,outputs,*RtrHi,Parse[Rtr],*Gesture[Contour],Parse[Atr],*AtrLo
0,atɪ,2,atɪ,1,0,0,0,0
1,atɪ,2,ətɪ,1,1,1,0,1
2,atɪ,2,ati,0,1,1,0,0
3,atɪ,2,əti,0,2,0,0,1


,input,winner,outputs,*RtrHi,Parse[Rtr],*Gesture[Contour],Parse[Atr],*AtrLo
0,atɛ,0,atɛ,0,0,0,0,0
1,atɛ,0,ətɛ,0,1,1,0,1
2,atɛ,0,ate,0,1,1,0,0
3,atɛ,0,əte,0,2,0,0,1


,input,winner,outputs,*RtrHi,Parse[Rtr],*Gesture[Contour],Parse[Atr],*AtrLo
0,ata,0,ata,0,0,0,0,0
1,ata,0,əta,0,1,1,0,1
2,ata,0,atə,0,1,1,0,1
3,ata,0,ətə,0,2,0,0,2


### Compute output probabilities

The following function accepts as input a weight vector $w$ and a tableau $T$, and returns maxent OT probabilities for all possible outputs in $T$, following Equation 1 in Goldwater and Johnson 2003:

\begin{equation}
\textrm{Pr}(y|x) \propto \exp(\sum_{i=1}^m w_i f_i(y,x))
\end{equation}

**NB**: In general we will expect negative weights $w_i$ because our "features" $f_i$ are constraint violations and we want constraint violations to count *against* an output, not for it.  If weights $w_i$ are negative, then an output $y$ with many constraint violations $f_i$ will yield a strongly negative (i.e. very low) value for $\sum_{i=1}^m w_i f_i(y,x)$, and correspondingly a low value for $\Pr(y|x)$, which is appropriate for an output that violates many constraints.

In [5]:
def tableau_prob(w,t):
    """
    function to compute maxent OT probabilities for all outputs in a tableau t.
    accepts as input:
    w: a weight vector, represented as a list
    t: a tableau, represented as a dataframe  
    and returns:
    p: probabilities for each output, represented as a list
    """
    # start by getting "features" i.e. constraint count violations (CVCs).
    # cvc is a list (by output) of lists (by feature) of CVCs.
    cvc = t[pdf_constraints].values.tolist()
    # score = numerator of eqn 1: exp of summed wts * features. 
    # scores = list of scores, one per output
    scores = [np.exp(np.dot(w,feature_vector)) for feature_vector in cvc]
    Z = sum(scores)   # denom
    p = scores / Z   # p is now a list of probs, one per output.
    return(p)

In [6]:
# now sanity-check the tableau_prob function by running it on input that is simple to understand.
# use the first tableau in the dataset, and an idealized weight vector of all -1s.
# NB: we use negative weights because we want constraint violations to count *against* an output,
# not for it.
t = tdf[0]
w = [-1,-1,-1,-1,-1]
display(t)
print(tableau_prob(w,t))
winner = t.winner[0]
print("winner is", winner)

,input,winner,outputs,*RtrHi,Parse[Rtr],*Gesture[Contour],Parse[Atr],*AtrLo
0,iti,0,iti,0,0,0,0,0
1,iti,0,ɪti,1,0,1,1,0
2,iti,0,itɪ,1,0,1,1,0
3,iti,0,ɪtɪ,2,0,0,2,0


[0.89454258 0.04453665 0.04453665 0.01638412]
winner is 0


### Was that the right answer?

Explain why or why not.  Show your intermediate results.

Numerators of G&J equation 1, for each output:
- 0: exp(0) = 1 
- 1: exp(-3) = 0.0497870684
- 2: exp(-3) = 0.0497870684
- 3: exp(-4) = 0.0183156389
- sum = 1.1178897756

Probabilities:
- 0: p = 0.894542576
- 1: p = 0.044536652
- 2: p = 0.044536652
- 3: p = 0.016384119

Yes, that matches the output we got.

Now double-sanity-check the function by doing that again but with a less trivial weight vector, of your own invention.  Again, do you get the right answer?  Explain why or why not.

### The maxent OT objective function

We will learn feature weights $w_i$ by adjusting them to optimize this function.  

The function accepts as input a weight vector $w$, a scalar $s2$ (for $\sigma^2$) that is used in regularization, and a "corpus" $c$ of tableaux.  It calculates the maxent OT objective function defined in Goldwater and Johnson 2003 equation 3:

\begin{equation}
\log \textrm{PL} (\bar{y}|\bar{x}) - \sum_{i=1}^m \frac{(w_i -\mu_i)^2}{2 \sigma_i^2}
\end{equation}

This is a difference of two terms.  The first term is the log of $\textrm{PL} (\bar{y}|\bar{x})$, defined in G&J equation 2 as:

\begin{equation}
\textrm{PL} (\bar{y}|\bar{x}) = \prod_{j=1}^n \textrm{Pr}(y_j^*|x_j)
\end{equation}

where $y_j^*$ denotes the designated winning output for tableau $j$.  Thus, maximizing this term means maximizing the probability of the intended winning output form, across tableaux.  The log of this quantity can be written as:

\begin{equation}
\log \textrm{PL} (\bar{y}|\bar{x}) = \log \prod_{j=1}^n \textrm{Pr}(y_j^*|x_j) =  \sum_{j=1}^n \log \textrm{Pr}(y_j^*|x_j)
\end{equation}

The second term is a *regularizing* term, intended to penalize weights of large magnitude.  Following G&J, we assume that $\mu_i = 0, \forall i$, and that $\sigma_i = \sigma, \forall i$.  Thus, there is only one free parameter to specify here, namely $\sigma$ or equivalently $\sigma^2$.


**NB**: Unlike G&J, we multiply the final result by -1.0 at the very end of the function code.  Why?  Because we want to maximize the objective function above, but our function will be given to scipy.optimize.minimize, which minimizes instead.  Whatever weights minimize $-x$ will maximize $x$.


In [7]:
def obj(w,s2,c):
    """
    maxent OT objective fn, based on G&J 2003 eqns 1,2,3.
    accepts as input
    w: a weight vector, one elt for each constraint
    s2: sigma^2 for regularization
    c: corpus of tableaus, represented as a list of dfs, to be unpacked into CVCs etc.
    to be maximized.  i multiply by -1.0 at the end bc this will
    be given to scipy.optimize.minimize - bc whatever param vals 
    minimize -1*x will maximize x.
    """
    # overall, objective = ll - reg, where
    # ll = log likelihood of winning output given input
    # reg = regularization term.
    
    # ll first.
    # ll = sum_(j = 1 to n) [log Pr (y^*_j | x_j)]
    # so to get ll, need Pr(y|x) for all outputs y for each given input x.
    # and to get that, we need to cycle through all tableaux in the corpus, 
    # bc each tableau corresponds to one input x.
    ll = 0.0
    for t in c:     # for tableau in corpus
        p = tableau_prob(w,t)
        # from here on we only care about prob of winner.
        winner = t.winner[0]  # find winner
        ll += np.log(p[winner])

    # regularization = sum_(i = 1 to m) (w_i - mu_i)^2 / 2*sigma_i^2
    # where m is number of features (constraints), and i ranges over them.
    # all mu_i are 0, and all sigma_i^2 are s2.
    # so regularization = sum_(i = 1 to m) w_i^2 / 2*sigma^2
    reg = sum([(wt**2)/(2.0*s2) for wt in w])
    
    objective = ll - reg
    return(-objective)    # minus bc we'll be minimizing and want to maximize.
    

### Optimize to find feature weights

Notice how easy this next part is!  It's good to be familiar with gradient descent, and to some extent with the many other variants on that theme that you can find in the [scipy.optimize documentation](https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html).  But you can control it all using a single function, [minimize](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html).  Feel free to explore different parameter settings for the minimize function call, to get a sense for how that affects the process and/or the product.

This optimization process takes a while to converge - it took about 10-15 seconds for me.

In [8]:
# set up input to minimize fn.
num_features = len(pdf_constraints)
w = [0.0]*num_features    # list of 0.0s.
s2 = 33333    # from G&J
# n = 10000  # hopefully not needed - G&J's number of iterations

# make your variable "corpus" be the corpus in list-of-dfs form.
corpus = tdf

# now optimize.
print("objective before minimization", obj(w,s2,corpus))
# CG = conjugate gradient, as in G&J.
res = minimize(obj, w,args=(s2,corpus), 
               # method='CG',
               options={'disp': True, 'maxiter': 10000}, 
               tol = 1.0E-6)
print("objective after minimization", res.fun)

objective before minimization 49.90659700031602
Optimization terminated successfully.
         Current function value: 0.039989
         Iterations: 74
         Function evaluations: 456
         Gradient evaluations: 76
objective after minimization 0.039988583439146595


### Examine resulting feature weights

How well do these match those shown in G&J Table 1, p. 4 of PDF?

In [9]:
w = res.x   # set to weights resulting from optimization
print("weights after training:")
for i in range(len(pdf_constraints)):
    print(pdf_constraints[i], ":", w[i])

weights after training:
*RtrHi : -39.73861408122175
Parse[Rtr] : -19.925278622075925
*Gesture[Contour] : -11.695629988941771
Parse[Atr] : -4.087183632164559
*AtrLo : -0.38853190294130124


### Is the right output always winning?

In [10]:
for t in tdf:
    probs = tableau_prob(w,t)
    winner = t.winner[0]
    print("probability of winner:", probs[winner])

probability of winner: 1.0
probability of winner: 0.9999998601665494
probability of winner: 0.9999997937716784
probability of winner: 0.9999999999999793
probability of winner: 0.9997334410738523
probability of winner: 0.9998192437841975
probability of winner: 0.9999998601665494
probability of winner: 0.9997180151738975
probability of winner: 0.9995842468351097
probability of winner: 0.9999998601248288
probability of winner: 0.999503871709104
probability of winner: 0.9995039142648362
probability of winner: 0.9999997937716784
probability of winner: 0.9995842468351097
probability of winner: 0.9993870602149608
probability of winner: 0.9999997937101579
probability of winner: 0.9996635456825802
probability of winner: 0.9996635745467838
probability of winner: 0.9999999999999793
probability of winner: 0.9999998601248288
probability of winner: 0.9999997937101578
probability of winner: 0.9999999999999586
probability of winner: 0.9994354767351397
probability of winner: 0.9995212283049841
probabil

# Observations and conclusions

- Write your general observations about the results here, and conclusions you can draw from them.  A short paragraph would be fine, but please separately specify observations and conclusions.

# Extensions (optional)

- Artificially adjust the Wolof data so they are stochastic rather than categorical, and test maxent OT on that adjusted data.  This is instead of running it on the (stochastic) Finnish data that G&J discuss, which I haven't obtained for you.